In [9]:
# importando as bibliotecas necessárias
import pandas as pd # manipulação de datasets
import numpy as np  # operações matemáticas 

# sklearn é uma biblioteca de data science e machine learning
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

from sklearn.model_selection import StratifiedKFold

from sklearn.model_selection import cross_validate
from sklearn import metrics

from sklearn import svm
import lightgbm as lgb

from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# biblioteca para visualização
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm import tqdm
import time

from sklearn.datasets import make_classification
from sklearn.utils import shuffle
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [3]:
X_lista = []
tests = [100, 1000, 10000, 100000, 1000000]

for i, test in enumerate(tests):
    X, y = make_classification(n_samples = 1000,
                               n_features = test,
                               n_informative = 20,
                               random_state=42,
                               shuffle=False
                               )
    X = pd.DataFrame(X, columns=range(len(X[0])))
    X = X[shuffle(list(X.columns), random_state=42)]
    X_lista.append(X)


In [ ]:
## MATRIZ DE CONFUSÃO
fig, axes = plt.subplots(1, 2, figsize=(12,6))

start = time.time()

y_pred = cross_val_predict(clf_lgbm, X, y, cv=skf)

end = time.time()
print('Tempo de execução: ', end-start)
cm_lgbm = confusion_matrix(y, y_pred)

#axes[0].set_title('Matriz Confusão SVM')
axes[0].set_title('Matriz Confusão LGBM')
#ConfusionMatrixDisplay(confusion_matrix=cm_svm).plot(ax = axes[0])
ConfusionMatrixDisplay(confusion_matrix=cm_lgbm).plot(ax = axes[0])


## MÉTRICAS
TP = (1,1)
TN = (0,0)
FP = (0,1)
FN = (1,0)
print('Métricas LGBM')
print(f"Acurácia: {round((cm_lgbm[TP] + cm_lgbm[TN])/(cm_lgbm[TP]+cm_lgbm[TN] + cm_lgbm[FP] + cm_lgbm[FN]),2)}")
print(f"Precisão: {round(cm_lgbm[TP] / (cm_lgbm[TP] + cm_lgbm[FP]),2)}")
print(f"Sensitivity: {round(cm_lgbm[TP]/(cm_lgbm[TP]+cm_lgbm[FN]),2)}")
print(f"Specificity: {round(cm_lgbm[TN]/(cm_lgbm[TN]+cm_lgbm[FP]),2)}")
print(f"F1: {round(2 * cm_lgbm[TP]/(2 * cm_lgbm[TP] + cm_lgbm[FN]+cm_lgbm[FP]),2)}")
print('----------------------------')

In [24]:
classifiers = []
for X in X_lista:
    start = time.time()
    clf_lgbm = lgb.LGBMClassifier(device_type='gpu', max_bin = 63)
    clf_lgbm.fit(X,y)
    classifiers.append(clf_lgbm)
    end = time.time()
    print("Tempo de execução: " + str(end-start) + " segundos")

[LightGBM] [Info] Number of positive: 499, number of negative: 501
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 6300
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 100
[LightGBM] [Info] Using GPU Device: TITAN Xp, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 64 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Info] Size of histogram bin entry: 8
[LightGBM] [Info] 100 dense feature groups (0.10 MB) transferred to GPU in 0.034332 secs. 0 sparse feature groups
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499000 -> initscore=-0.004000
[LightGBM] [Info] Start training from score -0.004000
Tempo de execução: 0.9456939697265625 segundos
[LightGBM] [Info] Number of positive: 499, number of negative: 501
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 63000
[LightGBM] [Info] Number of data points in the train set: 1000, number of used features: 1000


LightGBMError: Invalid Kernel Arguments

In [1]:

for classifier, X in zip(classifiers, X_lista):
    gabarito = pd.DataFrame({'Value':classifier.feature_importances_,'Feature':X.columns}).sort_values(by='Value', ascending = False).head(20).sort_values(by='Feature')
    gabarito['Correct'] = gabarito['Feature'].apply(lambda x: '✅' if x < 20 else '❌')
    display(gabarito)

for classifier in classifiers:
    lgb.plot_importance(classifier, max_num_features=30)


NameError: name 'classifiers' is not defined

0      1.952628
1     -0.788339
2     -2.075299
3     -3.015567
4     -0.375898
         ...   
995   -6.523888
996   -3.292432
997   -0.798333
998   -4.001240
999   -6.098424
Name: 0, Length: 1000, dtype: float64

In [22]:
X_new = SelectKBest(f_classif, k=20).fit(X_lista[1], y)
X_new.get_feature_names_out()

array(['x57', 'x70', 'x164', 'x259', 'x275', 'x298', 'x334', 'x374',
       'x420', 'x435', 'x482', 'x581', 'x600', 'x795', 'x915', 'x919',
       'x934', 'x979', 'x992', 'x995'], dtype=object)

In [19]:
X_lista[0]

,83,53,70,45,44,39,22,80,10,0,...,91,74,86,82,20,60,71,14,92,51
0,1.253932,-2.124445,0.059662,-0.171458,-0.706330,-0.439718,1.596089,-1.851657,-0.273657,1.952628,...,0.097041,0.512583,0.385527,-0.134615,2.866780,-0.254437,-1.814308,6.580329,0.336892,-0.309682
1,-0.132162,0.114913,0.431706,0.997351,0.146341,-2.045623,-0.128051,1.783230,0.116245,-0.788339,...,-0.301514,-0.861023,1.565821,-0.627477,2.076235,1.593094,-0.231408,-1.477229,-1.290368,-0.445677
2,-0.767547,-1.674570,0.234570,-0.446484,-1.079932,1.008752,-0.529113,1.353612,2.355029,-2.075299,...,-0.025411,-1.155076,1.442940,0.060736,12.185869,1.277619,0.601083,-0.197739,1.469213,-0.167835
3,0.155717,-1.549999,0.112332,0.057975,-1.600683,-0.693276,1.023433,1.193982,-1.662713,-3.015567,...,0.550304,0.148548,0.404748,-0.236547,10.718518,0.845271,1.511141,-0.065355,0.891079,-0.192278
4,-0.221133,0.358910,1.319101,-0.361512,-1.293626,-0.770314,-0.325660,-0.186613,-1.470090,-0.375898,...,-1.319729,0.588789,1.733883,0.921691,2.980072,0.181615,0.155575,1.173082,-0.131594,1.425910
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.168340,-0.449280,0.188648,-0.689339,2.261583,-0.553749,0.427247,-2.226031,0.534598,-6.523888,...,1.100064,1.020910,-1.007157,0.772546,-6.004473,1.325971,-0.773686,-5.269867,0.322415,2.130734
996,-2.055755,-0.281604,-0.377638,2.249280,-0.760063,-0.953757,1.188053,0.387761,-1.568665,-3.292432,...,-0.942118,1.751720,2.304103,-0.728670,-5.061490,-0.970105,-0.625364,-4.012976,1.657138,-0.374381
997,-0.637569,-0.776540,-0.787210,0.426073,0.137032,-0.108567,0.956172,-0.842893,1.028784,-0.798333,...,-0.407766,1.059522,-2.361084,-0.147617,-3.694466,1.124985,1.410989,1.215232,-0.599828,-0.467694
998,3.259767,0.407882,1.844511,0.439234,0.341038,0.607692,-1.928997,0.553706,-5.688534,-4.001240,...,-1.144848,-1.151800,0.791097,2.573798,-3.399175,-1.281604,-0.250024,-2.097617,-0.310686,0.388523
